# Sleep stage classification on Sleep-edf data set

In this project, we will use deep neural network on raw EEG data for sleep stage classification.

This work is based on the MNE-Python and the mne-torch repository, as well as

Chambon, S., Galtier, M. N., Arnal, P. J., Wainrib, G., & Gramfort, A. (2018). A deep learning architecture for temporal sleep stage classification using multivariate and multimodal time series. IEEE Transactions on Neural Systems and Rehabilitation Engineering, 26(4), 758-769.

# Sleep staging

The method of determining a person's sleep stage by studying their EEG and other physiological signs is known as sleep staging. Sleep recordings are generally separated into 30-second windows, with each window assigned to one of five groups ("stages"): <br>
1) W: wakefulness
2) N1: light sleep
3) N2: deeper sleep
4) N3: deep sleep
5) R: rapid eye movement
<br>

Sleep staging often relies on collecting changes in the EEG's spectral features as well as transitory events (such as sleep spindles, k-complexes, slow waves, and so on) that occur throughout various sleep phases.

We'll train a Deep neural network to conduct sleep staging on unseen raw EEG in this work. The Sleep Physionet dataset will be used, which comprises 153 nightly sleep recordings from 78 people. These recordings were carefully staged by sleep specialists, giving us the categorization objectives we needed to train and assess our ConvNet on

Identify whether a CUDA-enabled GPU is available

In [1]:

import torch

if torch.cuda.is_available():
    print('CUDA-enabled GPU found. Training should be faster.')
else:
    print('No GPU found. Training will be carried out on CPU, which might be '
          'slower.\n\nIf running on Google Colab, you can request a GPU runtime by'
          ' clicking\n`Runtime/Change runtime type` in the top bar menu, then '
          'selecting \'GPU\'\nunder \'Hardware accelerator\'.')
    

CUDA-enabled GPU found. Training should be faster.


# Install required packages if not installed
!pip install mne <br>
!pip install torch <br>
!pip install matplotlib <br>
!pip install scikit-learn <br>
!pip install pandas <br>